In [ ]:
# rm(list=ls())
require(data.table)
require(MASS)
require(ggplot2)
require(foreach)
require(gridExtra)
require(doMC)
registerDoMC(cores=16)

## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

In [2]:
## POP DATA from UN
# pop <- data.table(fread("/home/j/Project/IRH/Forecasting/data/WPP_forecasted_population.csv"))
pop <- fread('/ihme/forecasting/data/pop/20150101_wpp/data.csv')

In [22]:
## Education forecasts
edu_orig<-data.table(fread(paste0("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws.csv"), header = T))
edu_orig <- edu_orig[, V1:=NULL]
# edu<-data.table(fread(paste0("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/intermediate/gpr_draws_wide_by_years.csv"), header = T))
# edu <- data.table(edu)

Read 496860 rows and 1005 (of 1005) columns from 6.600 GB file in 00:01:48


In [23]:
## Convert age_group_id to age_start
edu_orig <- edu_orig[, age_group_id:=15+5*(age_group_id -8)] 
unique(edu_orig[,age_group_id])
head(edu_orig)

[1] 15 20 25 30 35 40 45 50 55 60 65 70 75 85

location_id,year_id,age_group_id,sex_id,draw0,draw1,draw2,draw3,draw4,draw5,⋯,draw990,draw991,draw992,draw993,draw994,draw995,draw996,draw997,draw998,draw999
6,1950,15,1,2.193899,3.696831,3.899294,5.886914,3.221693,3.998282,⋯,2.322269,3.697725,2.680337,3.931184,4.004720,4.729225,8.494130,2.570683,2.368506,3.664268
6,1951,15,1,2.329269,3.838159,4.045678,6.013686,3.346715,4.065710,⋯,2.427470,3.836150,2.780698,3.969950,4.142245,4.905914,8.606566,2.687016,2.487811,3.796609
6,1952,15,1,2.472498,3.986922,4.199735,6.134196,3.484166,4.133698,⋯,2.537848,3.977886,2.888934,4.009498,4.284303,5.089704,8.709071,2.806243,2.615149,3.934128
6,1953,15,1,2.623456,4.143215,4.361473,6.248749,3.634715,4.203069,⋯,2.653360,4.122637,3.005171,4.050488,4.431217,5.279684,8.801838,2.928341,2.751002,4.076550
6,1954,15,1,2.781863,4.307003,4.530767,6.357675,3.798821,4.274492,⋯,2.773895,4.270091,3.129476,4.093673,4.583275,5.474768,8.885073,3.053305,2.895764,4.223558
6,1955,15,1,2.947277,4.478090,4.707342,6.461299,3.976678,4.348449,⋯,2.899285,4.419948,3.261856,4.139855,4.740723,5.673722,8.958986,3.181143,3.049714,4.374800


In [24]:
### For now, let's concatenatate the age and sex columns to be one identifier:
system.time(edu_orig <- edu_orig[, age_sex := paste0(age_group_id, "_", sex_id)])
edu_orig <- edu_orig[, age_group_id:= NULL]
edu_orig <- edu_orig[, sex_id:= NULL]
edu_orig <- setcolorder(edu_orig, c("location_id", "year_id", "age_sex", paste0("draw",c(0:999)) ))
head(edu_orig)

   user  system elapsed 
  0.926   0.000   0.926 

location_id,year_id,age_sex,draw0,draw1,draw2,draw3,draw4,draw5,draw6,⋯,draw990,draw991,draw992,draw993,draw994,draw995,draw996,draw997,draw998,draw999
6,1950,15_1,2.193899,3.696831,3.899294,5.886914,3.221693,3.998282,3.933010,⋯,2.322269,3.697725,2.680337,3.931184,4.004720,4.729225,8.494130,2.570683,2.368506,3.664268
6,1951,15_1,2.329269,3.838159,4.045678,6.013686,3.346715,4.065710,4.017445,⋯,2.427470,3.836150,2.780698,3.969950,4.142245,4.905914,8.606566,2.687016,2.487811,3.796609
6,1952,15_1,2.472498,3.986922,4.199735,6.134196,3.484166,4.133698,4.106628,⋯,2.537848,3.977886,2.888934,4.009498,4.284303,5.089704,8.709071,2.806243,2.615149,3.934128
6,1953,15_1,2.623456,4.143215,4.361473,6.248749,3.634715,4.203069,4.201276,⋯,2.653360,4.122637,3.005171,4.050488,4.431217,5.279684,8.801838,2.928341,2.751002,4.076550
6,1954,15_1,2.781863,4.307003,4.530767,6.357675,3.798821,4.274492,4.302071,⋯,2.773895,4.270091,3.129476,4.093673,4.583275,5.474768,8.885073,3.053305,2.895764,4.223558
6,1955,15_1,2.947277,4.478090,4.707342,6.461299,3.976678,4.348449,4.409642,⋯,2.899285,4.419948,3.261856,4.139855,4.740723,5.673722,8.958986,3.181143,3.049714,4.374800


### Reshape long by draws, wide by year, use copula with spearman = 1, and reshape back AND collapse by country-year

In [25]:
# Make draws long first
system.time(edu_temp <- melt(edu_orig, id.vars = c("location_id", "age_sex", "year_id"), value.name = "edu", varnames = "draw"))
head(edu_temp)

   user  system elapsed 
  5.758   0.294   6.091 

location_id,age_sex,year_id,variable,edu
6,15_1,1950,draw0,2.193899
6,15_1,1951,draw0,2.329269
6,15_1,1952,draw0,2.472498
6,15_1,1953,draw0,2.623456
6,15_1,1954,draw0,2.781863
6,15_1,1955,draw0,2.947277


In [ ]:
## Tests
# test_pre_cop <- edu_temp[location_id==6 & age_group_id==8 & sex_id==2,]
# ggplot(data.frame(test_pre_cop)) + geom_line(aes(x = year_id, y = edu, group = variable, color=variable)) +
# theme(legend.position = "none")

In [ ]:
# Wide by year
# system.time(edu <- dcast(edu_temp , 
#                          formula = location_id + age_group_id + sex_id + variable ~ year_id, 
#                          value.var = "edu"))
# head(edu)

In [26]:
## Get our baseline correlation matrix
load("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/tabulated_data/insample_corr_mat.Rdata")

## For now, trim the corr matrix to have 14 age groups
corr_mat <- corr_mat[1:28,1:28]

In [ ]:
### Let's just work with one country for now
# edu_temp2 <- edu_temp[location_id == 6, ]

In [ ]:
# edu_temp2 <- edu_temp[order(variable, year_id, age_sex, location_id),]
# tail(edu_temp2)

In [ ]:
## List of stuff
# my_names <- list(location_id = unique(factor(edu_temp2[, location_id], ordered = T)),
#                  age_sex = unique(factor(edu_temp2[, age_sex], ordered = T)),
#                  year_id = unique(factor(edu_temp2[, year_id], ordered = T)),
#                  variable = unique(factor(edu_temp2[, variable], ordered = T)) )

In [ ]:
# print(c(length(my_names[[1]]), length(my_names[[2]]), length(my_names[[3]]), length(my_names[[4]])))

In [28]:
# Convert to array?
# myarray <- array(data=as.table(edu_temp2[, edu]), 
#                  c(length(my_names[[1]]), length(my_names[[2]]), length(my_names[[3]]), length(my_names[[4]])), 
#                  dimnames = my_names )

In [ ]:
# Convert to array?
system.time(myarray2<- reshape2::acast(edu_temp, location_id ~  year_id ~ age_sex ~ variable, value.var = "edu"))
str(myarray2)

In [35]:
### Subset out one country first
chn_array <- myarray2["6",,,]
str(chn_array)

 num [1:28, 1:91, 1:1000] 2.19 1.45 3.66 2.37 3.67 ...
 - attr(*, "dimnames")=List of 3
  ..$ : chr [1:28] "15_1" "15_2" "20_1" "20_2" ...
  ..$ : chr [1:91] "1950" "1951" "1952" "1953" ...
  ..$ : chr [1:1000] "draw0" "draw1" "draw2" "draw3" ...


### Neal's Univariate Copula Function

In [39]:
rank_time_series <- function(X, corr, ts=F, print=FALSE){
  draws <- nrow(X)
  years <- ncol(X)
  # X <- matrix(runif(draws*years), nrow=years, ncol=draws)
  
  if(print){
    print(sapply(2:years, function(x) cor(X[x,], X[x-1,])))
  }
  
  if(ts == F) {
    corr_mat <- matrix(data=corr, nrow= years, ncol= years )
    diag(corr_mat) <- 1
  }
  
  if(ts == T) {
    corr_mat <- corr**abs(outer(0:(years-1), 0:(years-1), "-"))
  }
  
  
#   print(corr_mat)
  mvdat <- mvrnorm(n= draws, mu=0 * 1:ncol(X), Sigma=corr_mat, empirical=T)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sorted_X <- apply(X, 2, sort)
  sapply(1:ncol(X), function(x) sorted_X[,x][ranks[,x]])
}



### Neal's Multivariate Copula Function

In [40]:
draw2Dcopula <- function(X, cor_mat){
    L <- dim(X)[2]
    D <- dim(X)[3]
    Xsum <- apply(X, c(2, 3), sum)
    mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
    ranks <- apply(mvdat, 2, rank, ties.method="first")
    sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
    sortedX <- Xtad
    for(i in 1:L){
        sortedX[,i,] <- Xtad[,i,sortedXsim[,i]]
    }
    Xcorr <- sortedX
    for(i in 1:L){
        Xcorr[,i,] <- sortedX[,i,ranks[,i]]
    }
    Xcorr
}

### Let's 2D copulate the data now

In [ ]:
Xcorr <- draw2Dcopula(Xtad, corr_mat)

# correlation over our first dimension (time) remains unchanged
sapply(1:ages, function(x) sapply(2:years, function(y)
  cor(Xcorr[y,x,], Xcorr[y-1,x,])))

# correlation over our second dimension (age) now approaches desired target
sapply(2:ages, function(x) sapply(1:years, function(y)
  cor(Xcorr[y,x,], Xcorr[y,x-1,])))

time_corr <- sapply(1:ages, function(x) sapply(2:years, function(y)
  cor(Xcorr[y,x,], Xcorr[y-1,x,])))

age_corr <- sapply(2:ages, function(x) sapply(1:years, function(y)   cor(Xcorr[y,x,], Xcorr[y,x-1,])))




In [ ]:
# ## Test for one case
# tesssst_1 <- rank_time_series(edu[location_id==102 & age_group_id==8 & sex_id==2,  6:96], corr = 1, ts = F,print=F)
# colnames(tesssst_1) <- paste0(c(1950:2040))
# tesssst_1 <- cbind(edu[location_id==6 & age_group_id==8 & sex_id==2,  1:5], tesssst_1)
# tesssst_2 <- data.table(melt(tesssst_1[, id :=NULL], id.vars=c("location_id", "age_group_id", "sex_id","variable"), value.name = "edu", variable.name = "year_id"))
# head(tesssst_1)

# ggplot(data.frame(tesssst_2)) + geom_line(aes(x=year_id, y = edu, group = variable, color=variable)) +
# theme(legend.position = "none")

### Use copula on all the data with Spearman = 1

In [ ]:
Start<- Sys.time()

alist<-list()
alist<- foreach(i = unique(edu[,id])) %dopar% {
    tempx <-  rank_time_series(edu[id==i, 6:96], corr = 1, ts = F,print=F)
    tempx <- cbind(edu[id==i, 1:5], tempx)
}


### Equivalent to the follow mclapply set up:

# test_func <- function(id_subset) {
#     results <- rank_time_series(edu[id==id_subset, 6:96], corr=1, ts=F, print=F)
#     results <- cbind(edu[id == id_subset, 1:5], results)
#     return(results)
# }

# test <- rbindlist(mclapply(unique(edu[,id]), test_func, mc.cores=32))

print(Sys.time()-Start)

edu_fill_cyas <- do.call(rbind, alist)
colnames(edu_fill_cyas) <- c("id", "location_id", "age_group_id", "sex_id", "variable", c(1950:2040))

In [ ]:
## Test plotss
# test_post_1 <- edu_fill_cyas[id == 2,]
# test_post_1 <- melt(test_post_1, id.vars = c("id", "location_id", "age_group_id", "sex_id", "variable"), value.name = "edu", variable.name = "year_id")
# head(test_post_1)

# ggplot(data.frame(test_post_1)) + geom_line(aes(x=year_id, y = edu, group = variable, color=variable)) +
# theme(legend.position = "none")

In [ ]:
# fwrite(edu_fill_cyas, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/intermediate/gpr_draws_wide_by_years_sorted.csv")

### Reshape to normal shape, and collapse to C-Y

In [ ]:
edu_fill_cyas <- edu_fill_cyas[, id:=NULL]
blank2_melt <- data.table(melt(edu_fill_cyas, id.vars = c("location_id", "age_group_id", "sex_id", "variable"), variable.name = "year_id", value.name = "data"))

In [ ]:
system.time(blank2_cast <- dcast(blank2_melt, location_id + age_group_id + sex_id + year_id ~ variable ))
head(blank2_cast)

In [ ]:
blank2_cast <- data.table(blank2_cast)

In [ ]:
head(blank2_cast)
plot_country_year_AS <- function(loc_id, age = 8, sex= 2) {
    return(ggplot(melt(blank2_cast[location_id==loc_id & age_group_id == age & sex_id == sex, ], 
                       id.vars = c("location_id", "age_group_id", "sex_id", "year_id"), 
                       value.name = "draw") ) + 
    geom_line(aes(x=year_id, y=draw, group=variable, color=variable)) + 
    scale_x_discrete(breaks = seq(1950,2040,by=5)) +
    theme(legend.position = "none"))    +
    ggtitle("Country-Year-Age-Sex Sorted")
    }

i <- 102
plot_country_year_AS(i)

In [ ]:
key(blank2_cast)

In [ ]:
system.time(fwrite(data.table(blank2_cast), "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_age_sex_post_copula.csv"))

In [ ]:
## Testing written CSV
# blank2_cast_test<- fread("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_age_sex_post_copula.csv", header=T)

In [ ]:
# head(blank2_cast_test)
# plot_country_year_AS_test <- function(loc_id, age = 8, sex= 1) {
#     return(ggplot(melt(blank2_cast_test[location_id==loc_id & age_group_id == age & sex_id == sex, ], 
#                        id.vars = c("location_id", "age_group_id", "sex_id", "year_id"), 
#                        value.name = "draw") ) + 
#     geom_line(aes(x=year_id, y=draw, group=variable, color=variable)) + 
#     scale_x_discrete(breaks = seq(1950,2040,by=5)) +
#     theme(legend.position = "none"))    +
#     ggtitle("Country-Year-Age-Sex Sorted")
#     }

# i <- 201
# plot_country_year_AS_test(i)

### Collapse to year and country only

In [ ]:
# blank2_cast <- fread("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_age_sex_post_copula.csv")
edu2 <- data.table(merge.data.frame(blank2_cast, pop, by = c("location_id", "year_id", "age_group_id", "sex_id")))
setkeyv(edu2, c("location_id", "year_id", "age_group_id", "sex_id"))

In [ ]:
edu4<- edu2[, lapply(.SD, function(x) sum(x*pop)/sum(pop)), 
    by=c("location_id", "year_id"), .SDcols=cbind(paste0("draw",c(0:999))) ] 

unique2 <- unique(edu4[, location_id])    
head(edu4)

In [ ]:
### PROBLEM: No population data for MHL (loc == 26) apparently (fix it)

### Copula this one? (uncomment if that's the intention)

In [ ]:
# edu4_1 <- melt(edu4[year_id >=1980,], id.vars = c("location_id", "year_id"), value.name = "edu", variable.name = "draw_num")
# edu4_2 <- data.table(dcast(edu4_1, location_id + draw_num ~ year_id, value.var = "edu"))
# head(edu4_2)

In [ ]:
# Start<- Sys.time()
# edu5 <- data.table()
# for(i in unique(edu4_2[,location_id])) {
#     temploc <- edu4_2[location_id==i, 1:2]
#     tempx <-  rank_time_series(edu4_2[location_id==i, 3:63], corr = 1, ts = F,print=F)
#     tempx <- cbind(temploc, tempx)
#     edu5 <- rbind(edu5, tempx)
# }

# colnames(edu5) <- c("location_id", "draw_", paste0(c(1980:2040)))
# # head(tempx)
                
# print(Sys.time()-Start)
# tail(edu5)

In [ ]:
# edu5_1 <- melt(edu5, id.vars = c("location_id", "draw_"), value.name = "edu", variable.name = "year_id")
# edu5_2 <- dcast(edu5_1, location_id + year_id ~ draw_, value.var = "edu")

In [ ]:
head(edu4)

In [ ]:
system.time(fwrite(edu4, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_collapsed_after_sort.csv"))
# fwrite(edu5_2, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_collapsed_after_sort.csv")

In [ ]:
# Test test test

plot_country_year <- function(loc_id) {
    return(ggplot(melt(data.table(edu4)[location_id==loc_id  , ], id.vars = c("location_id", "year_id"), value.name = "draw") ) + 
    geom_line(aes(x=year_id, y=draw, group=variable, color=variable)) + 
    scale_x_discrete(breaks = seq(1950,2040,by=5)) +
    theme(legend.position = "none"))+
    ggtitle("Country-Year Collapsed")
    }



In [ ]:
plot_country_year(6)